# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


**Описание данных**
* *id* — уникальный идентификатор скважины;
* *f0, f1, f2* — три признака точек (неважно, что они означают, но сами признаки значимы);
* *product* — объём запасов в скважине (тыс. баррелей).


Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных.

### Изучение предоставленных файлов

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import auc
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv') 
data2 = pd.read_csv('/datasets/geo_data_1.csv') 
data3 = pd.read_csv('/datasets/geo_data_2.csv')

***Выводим информацию о датафреймах.***


In [3]:
def all_info(data):
    print('-------------------------------------------------------------------------------------------------------------')
    print('Общая информация:')
    print(data.info())
    print('-----------------------------------------')
    print('Cтатистические данные по файлу:')
    print(data.describe())
    print('-----------------------------------------')
    print('Вывод первых пяти строк:')
    print(data.head())
    print('-----------------------------------------')
    print('Проверяем наличие дубликатов:')
    print(data.duplicated().sum())
    print('-----------------------------------------')
    print('Проверяем наличие дубликатов в id:')
    print(data['id'].duplicated().sum())
    print('------------------------------------------')
    print('Проверяем количество пропущенных значений:')
    print(data.isnull().sum())
    print('-------------------------------------------------------------------------------------------------------------')


***1. data1***

In [4]:
print( all_info(data1))

-------------------------------------------------------------------------------------------------------------
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------------
Cтатистические данные по файлу:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.

***2. data2***

In [5]:
print( all_info(data2))

-------------------------------------------------------------------------------------------------------------
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------------
Cтатистические данные по файлу:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.

***3. data3***

In [6]:
print(all_info(data3))

-------------------------------------------------------------------------------------------------------------
Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
-----------------------------------------
Cтатистические данные по файлу:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.

**Выводы:**
Размер выборок во всех датафреймах одинаковый: 100000 позиций, данные о которых распределены в 5 столбцах. О графах с признаками точек нет более полной информации, что-то сказать о данных в этих столбцах нельзя. Однако пропущенных значений в них нет, а нулевые значения сами по себе имеют место быть. В столбцах с уникальным идентификатором скважины встречаются дупликаты, однако в других столбцах под одним id данные разнятся, тк нет полных дубликатов. Будем считать эти строки измерениями одной скважины в разные моменты в времени, сделанные единично по какой-либо причине. В графе с объёмом запасов в скважине минимальное значение равно 0, что, в целом, можно допустить.

*Предобработка данных не требуется.*


## Обучение и проверка модели

### ***Разбиваем данные на обучающую и валидационную выборки для каждого региона:***

Разбиваем каждый датафрейм на обучающую и валидационную выборки в соотношении 75:25, т е 3:1.

В качестве целевого признака выделяем *'product'*

Так как данные в столбце с id имеют тип object, сохранение столбца будет приводить в дальнейшем к ошибке. Вместе с целевым признаков удаляем столбец с id, не сбрасывая индекс.

In [7]:
features_df1 = data1.drop(['id', 'product'], axis = 1) 
target_df1 = data1['product']

features_df2 = data2.drop(['id', 'product'], axis = 1)
target_df2 = data2['product']

features_df3 = data3.drop(['id', 'product'], axis = 1)
target_df3 = data3['product']



features_train_df1, features_valid_df1, target_train_df1, target_valid_df1 = train_test_split(
    features_df1, target_df1, test_size = 0.25, random_state = 12345)

features_train_df2, features_valid_df2, target_train_df2, target_valid_df2 = train_test_split(
    features_df2, target_df2, test_size = 0.25, random_state = 12345)

features_train_df3, features_valid_df3, target_train_df3, target_valid_df3 = train_test_split(
    features_df3, target_df3, test_size = 0.25, random_state = 12345)



Проверим размерность наборов: тренировочного и тестового:

In [8]:
print(target_train_df1.shape, target_valid_df1.shape)
print(target_train_df2.shape, target_valid_df2.shape)
print(target_train_df3.shape, target_valid_df3.shape)

(75000,) (25000,)
(75000,) (25000,)
(75000,) (25000,)


### Обучение и проверка модели

1. Обучите модель и сделайте предсказания на валидационной выборке.
2. Сохраните предсказания и правильные ответы на валидационной выборке.
3. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
4. Проанализируйте результаты.

**Месторождение 1**

In [9]:
model = LinearRegression()
model.fit(features_train_df1, target_train_df1) 
predictions_valid1 = model.predict(features_valid_df1) 
data1_pred_mean=predictions_valid1.mean()
result_df1 = mean_squared_error(target_valid_df1, predictions_valid1)**0.5
data1_predictions_valid = model.predict(features_valid_df1)

print("Средний предсказанный запас возможного сырья в месторождении валидационной выборки:", data1_pred_mean)
print("Средний фактический запас сырья в месторождении выборки:", target_valid_df1.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result_df1)

Средний предсказанный запас возможного сырья в месторождении валидационной выборки: 92.59256778438035
Средний фактический запас сырья в месторождении выборки: 92.07859674082927
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


**Месторождение 2**

In [10]:
model = LinearRegression()
model.fit(features_train_df2, target_train_df2) 
predictions_valid2 = model.predict(features_valid_df2) 
data2_pred_mean=predictions_valid2.mean()
result_df2 = mean_squared_error(target_valid_df2, predictions_valid2)**0.5
data2_predictions_valid = model.predict(features_valid_df2)


print("Средний предсказанный запас возможного сырья в месторождении валидационной выборки:", data2_pred_mean)
print("Средний фактический запас сырья в месторождении выборки:", target_valid_df2.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result_df2)

Средний предсказанный запас возможного сырья в месторождении валидационной выборки: 68.728546895446
Средний фактический запас сырья в месторождении выборки: 68.72313602435997
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


**Месторождение 3**

In [11]:
model = LinearRegression()
model.fit(features_train_df3, target_train_df3) 
predictions_valid3 = model.predict(features_valid_df3) 
data3_pred_mean=predictions_valid3.mean()
result_df3 = mean_squared_error(target_valid_df3, predictions_valid3)**0.5
data3_predictions_valid = model.predict(features_valid_df3)


print("Средний предсказанный запас возможного сырья в месторождении валидационной выборки:", data3_pred_mean )
print("Средний фактический запас сырья в месторождении выборки:", target_valid_df3.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result_df3)

Средний предсказанный запас возможного сырья в месторождении валидационной выборки: 94.96504596800489
Средний фактический запас сырья в месторождении выборки: 94.88423280885438
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


In [12]:
results = pd.DataFrame(
                       { "  ": ['Месторождение 1', 'Месторождение 2', 'Месторождение 3'],
                        "Средний предсказанный запас": [data1_pred_mean, data2_pred_mean,  data3_pred_mean],
                        "Средний фактический запас сырья": [target_valid_df1.mean(), target_valid_df2.mean(), target_valid_df3.mean()],
                        "RMSE модели  на валидационной выборке": [result_df1, result_df2, result_df3]})

results.set_index("  ")

,Средний предсказанный запас,Средний фактический запас сырья,RMSE модели на валидационной выборке
,,,
Месторождение 1,92.592568,92.078597,37.579422
Месторождение 2,68.728547,68.723136,0.893099
Месторождение 3,94.965046,94.884233,40.029709


### **Вывод:**
Сравнив полученные данные, можно сделать вывод, что судя по показателю RSME, самым выгодным явлеется второе месторождение, однако и по среднему предсказанному запасу сырья, и по фактическому оно уступает другим. Самый большой запас сырья в 3-м месторождении. 



## Подготовимся к расчёту прибыли.

1. Все ключевые значения для расчётов сохраним в отдельных переменных.
2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.
3. Напишем выводы по этапу подготовки расчёта прибыли.
4. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
5. Выберем скважины с максимальными значениями предсказаний.
6. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
7. Рассчитаем прибыль для полученного объёма сырья.

По условию, бюджет на разработку заложен в 10^10, доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. При разведке в регионе выбирают 200 лучших точек для разработки из выборки, равной 500.

Так же нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### ***Сформируем переменные и присвоим им вышеобозначенные значения.***

In [13]:
budget = 10 ** 10
revenue = 450000
count_borehole = 200

sufficient_volume = budget / revenue / count_borehole

print('Для безубыточной разработки необходимо чтобы с одной точки шла добыча не менее:', round(sufficient_volume + 1), 'тыс. баррелей \n')
print('В финансовом эквиваленте: ',   (sufficient_volume*revenue)/10**6     , 'млн')


Для безубыточной разработки необходимо чтобы с одной точки шла добыча не менее: 112 тыс. баррелей 

В финансовом эквиваленте:  50.0 млн


### ***Рассчитаем средний запас по каждому региону из изначальных файлов***

In [14]:
print('В среднем первый регион добывает:', round(data1['product'].mean()), 'тыс. баррелей')
print('В среднем второй регион добывает:', round(data2['product'].mean()), 'тыс. баррелей')
print('В среднем третий регион добывает:', round(data3['product'].mean()), 'тыс. баррелей')
print('Заложенный бюджет:', budget)

В среднем первый регион добывает: 93 тыс. баррелей
В среднем второй регион добывает: 69 тыс. баррелей
В среднем третий регион добывает: 95 тыс. баррелей
Заложенный бюджет: 10000000000


**Вывод:** исходя из средних цифр добычи по каждому месторождению, можено сказать, что большая часть их скважин не подходит под принцип безубыточной разработки. Т е при формировании итоговой выборки отклонение не должно быть большим, так как, в целом, под требования попадает относительно меньшее число скважин и нам нужно выбрать только наиболее прибыльные.

## Расчёт прибыли и рисков 

###  Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. 

Подготовим данные и сформируем функцию для расчета средней прибыли, 95% доверительного интервала(2,5% шаг) и риска убытков.

In [15]:
target_valid_df1 = target_valid_df1.reset_index(drop=True)
target_valid_df2 = target_valid_df2.reset_index(drop=True)
target_valid_df3 = target_valid_df3.reset_index(drop=True)
data1_predictions_valid = pd.Series(data1_predictions_valid)
data2_predictions_valid = pd.Series(data2_predictions_valid)
data3_predictions_valid = pd.Series(data3_predictions_valid)


In [16]:
def profit_function(target, predictions):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected_points = target[predictions_sorted.index][:200]
    product = selected_points.sum()
    revenue_all = product * revenue
    return revenue_all - budget


state = np.random.RandomState(1234)

def profit_risk(target, predictions):
    revenues = []
    for i in range(1000):
        target_sample = target.sample(n=500, replace = True, random_state = state)
        predictions_sample = predictions[target_sample.index]
        revenues.append(profit_function(target_sample, predictions_sample))
        
    revenues = pd.Series(revenues)
    
    lower = revenues.quantile(0.025)
    upper = revenues.quantile(0.975)
    profit = revenues.mean()
    risk = stats.percentileofscore(revenues, 0)
    return (lower, upper, profit, risk)




***Регион 1***

In [17]:
lower1, upper1, profit1, risk1 = profit_risk(target_valid_df1, data1_predictions_valid)
print("Средняя прибыль:", round(profit1/ (10 ** 6), 2), 'млн')
print("95%-й доверительный интервал(нижняя и верхняя границы соответственно):", (lower1, upper1))
print("Риск:", risk1)


Средняя прибыль: 417.82 млн
95%-й доверительный интервал(нижняя и верхняя границы соответственно): (-112839919.44204798, 964934886.248598)
Риск: 6.0


***Регион 2***

In [18]:
lower2, upper2, profit2, risk2 = profit_risk(target_valid_df2, data2_predictions_valid)
print("Средняя прибыль:", round(profit2/ (10 ** 6), 2), 'млн')
print("95%-й доверительный интервал(нижняя и верхняя границы соответственно):", (lower2, upper2))
print("Риск:", risk2)

Средняя прибыль: 528.87 млн
95%-й доверительный интервал(нижняя и верхняя границы соответственно): (118020423.53374214, 920140141.857394)
Риск: 0.3


***Регион 3***

In [19]:
lower3, upper3, profit3, risk3 = profit_risk(target_valid_df3, data3_predictions_valid)
print("Средняя прибыль:", round(profit3/ (10 ** 6), 2), 'млн')
print("95%-й доверительный интервал(нижняя и верхняя границы соответственно):", (lower3, upper3))
print("Риск:", risk3)

Средняя прибыль: 410.03 млн
95%-й доверительный интервал(нижняя и верхняя границы соответственно): (-97657713.7079339, 952816919.5140896)
Риск: 6.8


In [20]:
final_results = pd.DataFrame(
                       { "  ": ['Месторождение 1', 'Месторождение 2', 'Месторождение 3'],
                        "|Средняя прибыль, млн|": [round(profit1/ (10 ** 6), 2), round(profit2/ (10 ** 6), 2),  round(profit3/ (10 ** 6), 2)],
                        "|2,5% квантиль|": [lower1, lower2, lower3],
                        "|97.5 квантиль|": [upper1, upper2, upper3],
                        "|Риски, %|": [risk1, risk2, risk3]})

final_results.set_index("  ")

,"|Средняя прибыль, млн|","|2,5% квантиль|",|97.5 квантиль|,"|Риски, %|"
,,,,
Месторождение 1,417.82,-1.128399e+08,9.649349e+08,6.0
Месторождение 2,528.87,1.180204e+08,9.201401e+08,0.3
Месторождение 3,410.03,-9.765771e+07,9.528169e+08,6.8


###  **Вывод:**
Основываясь на полученных результатах можно сказать, что  наибольшую прибыль и минимальный риск показал второй регион.Но так же стоит учесть, несмотря на то, что по показателю RSME, самым выгодным явлеется второе месторождение, однако и по среднему предсказанному запасу сырья, и по фактическому оно уступает другим. Возможно, есть смысл расширить исследование. Далее из двух других регионов большую прибыль и наименьший риск имеет первый.

## ***Общий вывод***

В процессе работы были изучены предоставленные файлы: объем информации, статистические данные, информация о пропусках и дубликатах.  Далее были сформированы обучающие и валидационные выборки для каждого региона. В качестве целевого признака выделен 'product'. Была выполнена подготовка датафреймов. Далее методом линейной регрессии были обучены модели, информация была сохранена в соответствующие переменные. Были сревнены средний предсказанный запас возможного сырья и средний фактический запас сырья в месторождениях. Были выведены результаты RMSE модели линейной регрессии на валидационной выборке. По итогу можно сказать, что судя по показателю RSME, самым выгодным явлеется второе месторождение, однако и по среднему предсказанному запасу сырья, и по фактическому оно уступает другим. Самый большой запас сырья в 3-м месторождении. Далее вполнена подготовка к расчету прибыли: ключевые значения для расчётов сохранли в отдельных переменных; рассчитали достаточный объём сырья для безубыточной разработки новой скважины; сравнили полученный объём сырья со средним запасом в каждом регионе. Так же, исходя из средних цифр фактической добычи по каждому месторождению, можено сказать, что большая часть их скважин не подходит под принцип безубыточной разработки. Т е при формировании итоговой выборки отклонение не должно быть большим, так как, в целом, под требования попадает относительно меньшее число скважин и нам нужно выбрать только наиболее прибыльные. Рассчитали прибыль для полученного объёма сырья.

*Основываясь на полученных результатах можно сказать, что наибольшую прибыль и минимальный риск показал **второй регион**, принимаем его за рекомендованный к разработке.* Но так же стоит учесть, несмотря на то, что по показателю RSME, самым выгодным явлеется второе месторождение, однако и по среднему предсказанному запасу сырья, и по фактическому оно уступает другим. Возможно, есть смысл расширить исследование. Далее из двух других регионов большую прибыль и наименьший риск имеет первый.
